# Лабораторная 1
## Подраздел: LU-разложение

* Cтудент: Ефимов А.В.
* Группа: М8О-307Б
* Вариант: 7

Загрузка библиотек, матрицы

In [1]:
import numpy as np
import logging, json
from math import isclose

# Configure
logging.basicConfig(level=logging.DEBUG)

# Load matrix
with open("task.json", "r") as json_file:
    task = json.load(json_file)

matrix = np.array(task["matrix"])
constants = np.array(task["constants"])


Функция разложения матрицы на нижнюю и верхнуюю треугольные, выполняющаяся методом Гаусса:

In [3]:
def decompose_matrix(matrix):
    """Decomposes the matrix into lower and upper matrices"""
    if matrix.shape[0] != matrix.shape[1]:
        raise IndexError("Recieved matrix is not square")
    
    logging.info(f"Input matrix:\n{matrix}")
    
    matrix_size = len(matrix)
    lower = np.zeros_like(matrix, dtype=np.float32)
    upper = np.array(matrix, dtype=np.float32)

    permut = np.arange(matrix_size + 1)
    permut[matrix_size] = 0

    for iter_i in range(matrix_size):
        logging.debug(f"Iteration no. {iter_i}")
        max_i = iter_i
        max_val = abs(matrix[iter_i, iter_i])

        for i in range(iter_i + 1, matrix_size):
            abs_val = abs(matrix[i, iter_i])
            if abs_val > max_val:
                max_val = abs_val
                max_i = i

        logging.debug(f"Max value on row {max_i}: {max_val}")
        if isclose(max_val, 0, abs_tol=10e-5):
            raise RuntimeError("Recieved matrix is degenerate")

        if iter_i != max_i:
            permut[[max_i, iter_i]] = permut[[iter_i, max_i]]
            upper[[max_i, iter_i]] = upper[[iter_i, max_i]]
            lower[[max_i, iter_i]] = lower[[iter_i, max_i]]
            permut[matrix_size] += 1

        lower[iter_i, iter_i] = 1
        lower[iter_i+1:, iter_i] = (upper[iter_i+1:, iter_i] / upper[iter_i, iter_i])
        
        upper[iter_i+1:] = upper[iter_i+1:] - np.outer(lower[iter_i+1:, iter_i], upper[iter_i])
        
        logging.debug(f"Current upper:\n{upper}")
        logging.debug(f"Current lower:\n{lower}")
        logging.debug(f"Current permutations: {permut}")
    
    logging.info("Decomposition finished")
    
    return (lower, upper, permut)

lower, upper, permutation = decompose_matrix(matrix)
print(f"Upper:\n{upper}")
print(f"Lower:\n{lower}")
print(f"Permutations: {permutation}")


INFO:root:Input matrix:
[[ 1 -5 -7  1]
 [ 1 -3 -9 -4]
 [-2  4  2  1]
 [-9  9  5  3]]
DEBUG:root:Iteration no. 0
DEBUG:root:Max value on row 3: 9
DEBUG:root:Current upper:
[[-9.          9.          5.          3.        ]
 [ 0.         -2.         -8.444445   -3.6666667 ]
 [ 0.          2.          0.88888884  0.3333333 ]
 [ 0.         -4.         -6.4444447   1.3333334 ]]
DEBUG:root:Current lower:
[[ 1.          0.          0.          0.        ]
 [-0.11111111  0.          0.          0.        ]
 [ 0.22222222  0.          0.          0.        ]
 [-0.11111111  0.          0.          0.        ]]
DEBUG:root:Current permutations: [3 1 2 0 1]
DEBUG:root:Iteration no. 1
DEBUG:root:Max value on row 3: 9
DEBUG:root:Current upper:
[[-9.         9.         5.         3.       ]
 [ 0.        -4.        -6.4444447  1.3333334]
 [ 0.         0.        -2.3333335  1.       ]
 [ 0.         0.        -5.2222223 -4.3333335]]
DEBUG:root:Current lower:
[[ 1.          0.          0.          0.      

Upper:
[[-9.         9.         5.         3.       ]
 [ 0.        -4.        -6.4444447  1.3333334]
 [ 0.         0.        -5.2222223 -4.3333335]
 [ 0.         0.         0.         2.9361706]]
Lower:
[[ 1.          0.          0.          0.        ]
 [-0.11111111  1.          0.          0.        ]
 [-0.11111111  0.5         1.          0.        ]
 [ 0.22222222 -0.5         0.44680855  1.        ]]
Permutations: [3 0 1 2 3]


Решение системы с помощью разложенной матрицы через системы $Lz = b$ и $Ux = z$

In [6]:
def solve_lu(lower, upper, permutation, constants):
    """Solves the LU decompostion with given constants 
    (equation constraints)"""
    matrix_size = len(lower)

    logging.info(f"Input constants: {constants}")
    
    constants = constants[permutation[:-1]]

    logging.info(f"Permuted constants: {constants}")
    
    temp_constants = np.zeros_like(constants, dtype=np.float32)
    for i in range(matrix_size):
        for j in range(i):
            temp_constants[i] -= temp_constants[j] * lower[i][j]
        temp_constants[i] = (temp_constants[i] + constants[i]) / lower[i][i]

    logging.debug(f"z = {temp_constants}")
    
    result = np.zeros_like(constants, dtype=np.float32)
    for i in reversed(range(matrix_size)):
        for j in range(i + 1, matrix_size):
            result[i] -= result[j] * upper[i][j]
        result[i] = (result[i] + temp_constants[i]) / upper[i][i]

    logging.info(f"Solution: {result}")
        
    return result

x_vals = solve_lu(lower, upper, permutation, constants)
print(f"Solution: {x_vals}")

INFO:root:Input constants: [-75 -41  18  29]
INFO:root:Permuted constants: [ 29 -75 -41  18]
DEBUG:root:z = [ 29.        -71.77778    -1.8888893 -23.489365 ]
INFO:root:Solution: [ 1.9999992  3.999999   7.        -8.       ]


Solution: [ 1.9999992  3.999999   7.        -8.       ]


Через разложение можно найти определитель (при этом учитывая, что одна перестановка меняла знак на отрицательный, а две - самоуничтожались):

In [9]:
def calc_lu_determinant(lower, upper, p):
    """Calculates determinant from LU decomposition. 
    Permutation array is necessary to calculate sign"""
    result = 1
    
    matrix_size = len(lower)

    for i in range(matrix_size):
        result *= lower[i][i] * upper[i][i]

    result *= 1 if (p[matrix_size] % 2 == 0) else -1

    return result

print(f"LU determinant: {calc_lu_determinant(lower, upper, permutation)}")
print(f"Numpy determinant: {np.linalg.det(matrix)}")

LU determinant: 552.0000798651527
Numpy determinant: 551.9999999999998


Наконец, с помощью разложения можно посчитать обратную разложенной матрицы просто
решая примыкающую единичную матрицу по столбцам:

In [10]:
def calc_lu_inverse(lower, upper, p):
    """Calculates columns of matrice inverse from its decomposition.
    Transposition ensures the same direction of lists as original"""
    matrix_size = len(lower)
    result = np.zeros_like(lower, dtype=np.float32)
    for i in range(matrix_size):
        inverse_column = np.zeros((matrix_size, ))
        inverse_column[i] = 1
        result[i] = solve_lu(lower, upper, p, inverse_column)

    return result.T

inverse = calc_lu_inverse(lower, upper, permutation)
print(f"Inverse:\n{inverse}")
print(f"Multiplication result:\n{matrix @ inverse}")

INFO:root:Input constants: [1. 0. 0. 0.]
INFO:root:Permuted constants: [0. 1. 0. 0.]
DEBUG:root:z = [ 0.         1.        -0.5        0.7234043]
INFO:root:Solution: [ 0.02898551  0.00724638 -0.10869564  0.2463768 ]
INFO:root:Input constants: [0. 1. 0. 0.]
INFO:root:Permuted constants: [0. 0. 1. 0.]
DEBUG:root:z = [ 0.          0.          1.         -0.44680855]
INFO:root:Solution: [-0.0326087   0.05434782 -0.06521738 -0.1521739 ]
INFO:root:Input constants: [0. 0. 1. 0.]
INFO:root:Permuted constants: [0. 0. 0. 1.]
DEBUG:root:z = [0. 0. 0. 1.]
INFO:root:Solution: [ 0.52536225  0.5688405  -0.28260866  0.34057966]
INFO:root:Input constants: [0. 0. 0. 1.]
INFO:root:Permuted constants: [1. 0. 0. 0.]
DEBUG:root:z = [ 1.          0.11111111  0.05555556 -0.19148937]
INFO:root:Solution: [-0.22826086 -0.1195652   0.04347825 -0.06521738]


Inverse:
[[ 0.02898551 -0.0326087   0.52536225 -0.22826086]
 [ 0.00724638  0.05434782  0.5688405  -0.1195652 ]
 [-0.10869564 -0.06521738 -0.28260866  0.04347825]
 [ 0.2463768  -0.1521739   0.34057966 -0.06521738]]
Multiplication result:
[[ 9.99999918e-01 -7.45058060e-09  0.00000000e+00 -3.72529030e-09]
 [-3.72529030e-08  9.99999925e-01  2.98023224e-08 -3.72529030e-09]
 [ 0.00000000e+00 -7.45058060e-09  9.99999851e-01  2.98023224e-08]
 [-7.45058060e-09  7.45058060e-09 -5.96046448e-08  1.00000003e+00]]
